# Comparing median income and crime in chicago


In [1]:
import csv
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 
import numpy as np 

### Store CSV into DataFrame

In [2]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
crime_data = pd.read_csv("Crimes_-_2001_to_Present.csv", encoding="ISO-8859-1")

In [3]:
#Drop rows with null values
crime_data.dropna()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
63300,11665567,JC234307,04/10/2019 04:37:00 PM,102XX S VERNON AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,SCHOOL - PUBLIC BUILDING,False,False,...,9.0,49.0,17,1181051.0,1837225.0,2019,04/17/2020 03:50:24 PM,41.708589,-87.612583,"(41.708589, -87.612583094)"
63384,11667963,JC235212,04/12/2019 04:08:00 PM,032XX N KEELER AVE,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,True,...,30.0,16.0,02,1147835.0,1921408.0,2019,10/02/2019 04:13:24 PM,41.940298,-87.732066,"(41.940297617, -87.732066473)"
63385,11667968,JC237058,04/19/2019 01:57:00 PM,002XX N LARAMIE AVE,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,28.0,25.0,17,1141669.0,1901165.0,2019,03/18/2020 03:52:17 PM,41.884865,-87.755230,"(41.884865037, -87.755230327)"
63461,11668309,JC238187,04/25/2019 05:20:00 PM,108XX S DR MARTIN LUTHER KING JR DR,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,9.0,49.0,08B,1180832.0,1833222.0,2019,06/30/2019 03:56:27 PM,41.697609,-87.613508,"(41.697609261, -87.613507612)"
63865,11692179,JC261724,05/13/2019 05:26:00 PM,090XX S RACINE AVE,0560,ASSAULT,SIMPLE,STREET,False,False,...,21.0,73.0,08A,1169908.0,1844927.0,2019,06/30/2019 03:56:27 PM,41.729973,-87.653167,"(41.729973132, -87.653166753)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7146131,9999999,HY189683,03/18/2015 05:08:00 PM,043XX N CENTRAL AVE,0560,ASSAULT,SIMPLE,RESTAURANT,True,False,...,38.0,15.0,08A,1138222.0,1928502.0,2015,02/10/2018 03:50:01 PM,41.959944,-87.767226,"(41.959943835, -87.767225719)"
7146132,11700037,JC278815,05/26/2019 12:10:00 AM,046XX W MC LEAN AVE,0460,BATTERY,SIMPLE,RESIDENCE,True,False,...,36.0,19.0,08B,1145043.0,1913219.0,2019,06/30/2019 03:56:27 PM,41.917879,-87.742535,"(41.917879493, -87.742535496)"
7146134,11700926,JC279725,05/26/2019 05:13:00 PM,036XX W DOUGLAS BLVD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,False,True,...,24.0,29.0,26,1152126.0,1893208.0,2019,06/30/2019 03:56:27 PM,41.862830,-87.717040,"(41.862830429, -87.717040084)"
7146135,24560,JC279072,05/26/2019 06:48:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,CHA PARKING LOT,True,False,...,25.0,28.0,01A,1167752.0,1893853.0,2019,06/20/2020 03:48:45 PM,41.864278,-87.659660,"(41.864278228, -87.659660218)"


In [4]:
crime_data.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,7.146139e+06,7.146139e+06,7.146092e+06,6.531312e+06,6.532655e+06,7.077300e+06,7.077300e+06,7.146139e+06,7.077300e+06,7.077300e+06
mean,6.534422e+06,1.188921e+03,1.129473e+01,2.270921e+01,3.755227e+01,1.164544e+06,1.885727e+06,2.008922e+03,4.184203e+01,-8.767170e+01
std,3.218577e+06,7.030735e+02,6.945838e+00,1.383348e+01,2.153480e+01,1.712851e+04,3.265277e+04,5.463675e+00,8.985009e-02,6.199727e-02
min,6.340000e+02,1.110000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,3.661945e+01,-9.168657e+01
25%,3.555666e+06,6.220000e+02,6.000000e+00,1.000000e+01,2.300000e+01,1.152947e+06,1.859132e+06,2.004000e+03,4.176883e+01,-8.771379e+01
50%,6.525852e+06,1.111000e+03,1.000000e+01,2.200000e+01,3.200000e+01,1.166036e+06,1.890649e+06,2.008000e+03,4.185573e+01,-8.766604e+01
75%,9.322159e+06,1.731000e+03,1.700000e+01,3.400000e+01,5.700000e+01,1.176358e+06,1.909232e+06,2.013000e+03,4.190672e+01,-8.762831e+01
max,1.209601e+07,2.535000e+03,3.100000e+01,5.000000e+01,7.700000e+01,1.205119e+06,1.951622e+06,2.020000e+03,4.202291e+01,-8.752453e+01


In [5]:
#Add new Year column
crime_data['Year'] = crime_data.apply(lambda row: row.Date[6:10], axis = 1)
crime_data

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7146134,11700926,JC279725,05/26/2019 05:13:00 PM,036XX W DOUGLAS BLVD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,APARTMENT,False,True,...,24.0,29.0,26,1152126.0,1893208.0,2019,06/30/2019 03:56:27 PM,41.862830,-87.717040,"(41.862830429, -87.717040084)"
7146135,24560,JC279072,05/26/2019 06:48:00 AM,013XX W HASTINGS ST,0110,HOMICIDE,FIRST DEGREE MURDER,CHA PARKING LOT,True,False,...,25.0,28.0,01A,1167752.0,1893853.0,2019,06/20/2020 03:48:45 PM,41.864278,-87.659660,"(41.864278228, -87.659660218)"
7146136,11707734,JC287730,07/01/2014 07:30:00 AM,063XX S NORMAL BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,20.0,68.0,11,NaN,NaN,2014,06/02/2019 04:09:42 PM,NaN,NaN,NaN
7146137,11707239,JC287563,11/30/2017 09:00:00 AM,022XX S KOSTNER AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,22.0,29.0,11,NaN,NaN,2017,06/02/2019 04:09:42 PM,NaN,NaN,NaN


In [6]:
# convert year to numeric
crime_data["Year"] = pd.to_numeric(crime_data["Year"])

##  Filter for years 2011 to 2018

In [8]:
#Filter for on 2011 to 2018 years
crime_data = crime_data[crime_data['Year'].between(2011, 2018)]
crime_data

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,...,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,...,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,...,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,...,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
5,11227517,JB138481,02/10/2013 12:00:00 AM,071XX S LAFAYETTE AVE,0266,CRIM SEXUAL ASSAULT,PREDATORY,RESIDENCE,False,False,...,6.0,69.0,02,NaN,NaN,2013,02/11/2018 03:57:41 PM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7146129,9999997,HY189887,03/18/2015 07:55:00 PM,032XX W HURON ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE PORCH/HALLWAY,False,True,...,27.0,23.0,08B,1154760.0,1904484.0,2015,02/10/2018 03:50:01 PM,41.893721,-87.707069,"(41.893720636, -87.707068862)"
7146130,9999998,HY188874,03/18/2015 04:00:00 AM,036XX W BELMONT AVE,0460,BATTERY,SIMPLE,RESIDENCE,False,False,...,35.0,21.0,08B,1151659.0,1921038.0,2015,02/10/2018 03:50:01 PM,41.939208,-87.718022,"(41.939207846, -87.718021657)"
7146131,9999999,HY189683,03/18/2015 05:08:00 PM,043XX N CENTRAL AVE,0560,ASSAULT,SIMPLE,RESTAURANT,True,False,...,38.0,15.0,08A,1138222.0,1928502.0,2015,02/10/2018 03:50:01 PM,41.959944,-87.767226,"(41.959943835, -87.767225719)"
7146136,11707734,JC287730,07/01/2014 07:30:00 AM,063XX S NORMAL BLVD,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,20.0,68.0,11,NaN,NaN,2014,06/02/2019 04:09:42 PM,NaN,NaN,NaN


## Random Sample of 500,000 rows of data

In [9]:
# Random sample of 500,000 rows of data from 7 millions rows of data
sample_crime_data = crime_data.sample(n = 500000) 
sample_crime_data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
522244,10298173,HY486074,10/22/2012 12:01:00 AM,013XX N WOLCOTT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,1.0,24.0,11,1163654.0,1908992.0,2012,02/10/2018 03:50:01 PM,41.905908,-87.674277,"(41.905908138, -87.674277024)"
6784828,9383887,HW526874,11/08/2013 10:00:00 AM,025XX E 78TH ST,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE-GARAGE,False,False,...,7.0,43.0,05,1194390.0,1853738.0,2013,02/10/2018 03:50:01 PM,41.753585,-87.563194,"(41.753585384, -87.563194315)"
613539,10456276,HZ195619,03/17/2016 08:00:00 PM,032XX N CICERO AVE,1320,CRIMINAL DAMAGE,TO VEHICLE,PARKING LOT/GARAGE(NON.RESID.),False,False,...,30.0,15.0,14,1143808.0,1920986.0,2016,02/10/2018 03:50:01 PM,41.939216,-87.746878,"(41.939216164, -87.746877814)"
1319402,11506866,JB515331,11/12/2018 08:00:00 AM,077XX S HARPER AVE,0810,THEFT,OVER $500,VACANT LOT/LAND,False,False,...,5.0,43.0,06,1187883.0,1853844.0,2018,11/19/2018 04:22:02 PM,41.754034,-87.587036,"(41.754033621, -87.587036374)"
6545627,8997819,HW145057,11/22/2012 12:01:00 AM,031XX W LELAND AVE,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,True,...,33.0,14.0,26,1154428.0,1931054.0,2012,02/10/2018 03:50:01 PM,41.966637,-87.707576,"(41.966637431, -87.707575998)"


In [10]:
# Description of crime data
sample_crime_data.describe()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,5.000000e+05,500000.000000,499999.000000,499992.000000,499963.000000,4.955310e+05,4.955310e+05,500000.000000,495531.000000,495531.000000
mean,9.736921e+06,1155.756804,11.245862,22.946833,37.335405,1.164534e+06,1.885711e+06,2014.271508,41.841988,-87.671738
std,1.170559e+06,695.003381,6.920288,13.848570,21.440461,1.739655e+04,3.277703e+04,2.329952,0.090200,0.062937
min,1.973600e+04,111.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,2011.000000,36.619446,-91.686566
25%,8.788834e+06,613.000000,6.000000,10.000000,23.000000,1.152753e+06,1.858876e+06,2012.000000,41.768131,-87.714524
50%,9.747599e+06,1024.000000,10.000000,23.000000,32.000000,1.166194e+06,1.891719e+06,2014.000000,41.858568,-87.665680
75%,1.073866e+07,1713.000000,17.000000,34.000000,56.000000,1.176369e+06,1.908781e+06,2016.000000,41.905468,-87.628056
max,1.209560e+07,2535.000000,31.000000,50.000000,77.000000,1.205119e+06,1.951535e+06,2018.000000,42.022671,-87.524529


## Check for types of crime

In [11]:
uniq_crime_type = sample_crime_data['Primary Type'].unique() 
uniq_crime_type

# crime_data.columns

# Description

array(['DECEPTIVE PRACTICE', 'BURGLARY', 'CRIMINAL DAMAGE', 'THEFT',
       'OTHER OFFENSE', 'BATTERY', 'WEAPONS VIOLATION',
       'MOTOR VEHICLE THEFT', 'NARCOTICS', 'ASSAULT',
       'INTERFERENCE WITH PUBLIC OFFICER', 'OFFENSE INVOLVING CHILDREN',
       'CRIMINAL TRESPASS', 'ROBBERY', 'NON-CRIMINAL',
       'CRIM SEXUAL ASSAULT', 'HOMICIDE', 'PUBLIC PEACE VIOLATION',
       'GAMBLING', 'CONCEALED CARRY LICENSE VIOLATION', 'PROSTITUTION',
       'HUMAN TRAFFICKING', 'LIQUOR LAW VIOLATION', 'OBSCENITY',
       'SEX OFFENSE', 'ARSON', 'CRIMINAL SEXUAL ASSAULT', 'INTIMIDATION',
       'KIDNAPPING', 'STALKING', 'NON-CRIMINAL (SUBJECT SPECIFIED)',
       'NON - CRIMINAL', 'PUBLIC INDECENCY', 'OTHER NARCOTIC VIOLATION'],
      dtype=object)

In [12]:
#Check for FBI codes
uniq_crime_desc = crime_data['FBI Code'].unique() 
uniq_crime_desc

array(['02', '05', '06', '17', '11', '14', '26', '20', '12', '08A', '18',
       '10', '03', '13', '15', '07', '08B', '04B', '04A', '22', '16',
       '24', '09', '19', '01A', '01B'], dtype=object)

## Added new column "Crime" with FBI Uniform crime report


In [13]:
# Added column "Crime" with FBI Uniform crime report
sample_crime_data.loc[sample_crime_data['FBI Code'] == '01A', 'Crime'] = ' Murder and Nonnegligent Manslaughter'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '01B', 'Crime'] = 'Manslaughter by Negligence'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '02', 'Crime'] = 'Forcible Rape'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '03', 'Crime'] = 'Robbery'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '04A', 'Crime'] = 'Aggravated Assaul- Firearm'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '04B', 'Crime'] = 'Aggravated Assaul-Knife or Cutting Instrument'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '05', 'Crime'] = 'Burglary'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '06', 'Crime'] = 'Larceny-thefT'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '07', 'Crime'] = 'Motor Vehicle Thef'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '08A', 'Crime'] = 'Arson-Structural'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '08B', 'Crime'] = 'Arson-Structural'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '09', 'Crime'] = 'Other Assaults'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '10', 'Crime'] = 'Forgery and Counterfeiting'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '12', 'Crime'] = 'Embezzlement'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '13', 'Crime'] = 'Stolen Property: Buying, Receiving, Possessing'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '14', 'Crime'] = 'Vandalism'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '15', 'Crime'] = 'Weapons: Carrying, Possessing, etc'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '16', 'Crime'] = ' Prostitution and Commercialized Vice'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '17', 'Crime'] = 'Sex Offenses'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '18', 'Crime'] = 'Drug Abuse Violations'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '19', 'Crime'] = 'Gambling'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '20', 'Crime'] = 'Offenses Against the Family and Children'
sample_crime_data.loc[sample_crime_data['FBI Code'] == '26', 'Crime'] = 'All Other Offenses'


In [14]:
#Merged Battery and Assault crimes
sample_crime_data.loc[sample_crime_data['Primary Type'] == 'BATTERY', 'Primary Type'] = 'ASSAULT'

In [15]:
#Unique crime types
uniq_crime_type = sample_crime_data['Primary Type'].unique() 
uniq_crime_type

array(['DECEPTIVE PRACTICE', 'BURGLARY', 'CRIMINAL DAMAGE', 'THEFT',
       'OTHER OFFENSE', 'ASSAULT', 'WEAPONS VIOLATION',
       'MOTOR VEHICLE THEFT', 'NARCOTICS',
       'INTERFERENCE WITH PUBLIC OFFICER', 'OFFENSE INVOLVING CHILDREN',
       'CRIMINAL TRESPASS', 'ROBBERY', 'NON-CRIMINAL',
       'CRIM SEXUAL ASSAULT', 'HOMICIDE', 'PUBLIC PEACE VIOLATION',
       'GAMBLING', 'CONCEALED CARRY LICENSE VIOLATION', 'PROSTITUTION',
       'HUMAN TRAFFICKING', 'LIQUOR LAW VIOLATION', 'OBSCENITY',
       'SEX OFFENSE', 'ARSON', 'CRIMINAL SEXUAL ASSAULT', 'INTIMIDATION',
       'KIDNAPPING', 'STALKING', 'NON-CRIMINAL (SUBJECT SPECIFIED)',
       'NON - CRIMINAL', 'PUBLIC INDECENCY', 'OTHER NARCOTIC VIOLATION'],
      dtype=object)

In [16]:
# Describe filtered data 
sample_crime_data.describe()

# sample_crime_data.head()

,ID,Beat,District,Ward,Community Area,X Coordinate,Y Coordinate,Year,Latitude,Longitude
count,5.000000e+05,500000.000000,499999.000000,499992.000000,499963.000000,4.955310e+05,4.955310e+05,500000.000000,495531.000000,495531.000000
mean,9.736921e+06,1155.756804,11.245862,22.946833,37.335405,1.164534e+06,1.885711e+06,2014.271508,41.841988,-87.671738
std,1.170559e+06,695.003381,6.920288,13.848570,21.440461,1.739655e+04,3.277703e+04,2.329952,0.090200,0.062937
min,1.973600e+04,111.000000,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,2011.000000,36.619446,-91.686566
25%,8.788834e+06,613.000000,6.000000,10.000000,23.000000,1.152753e+06,1.858876e+06,2012.000000,41.768131,-87.714524
50%,9.747599e+06,1024.000000,10.000000,23.000000,32.000000,1.166194e+06,1.891719e+06,2014.000000,41.858568,-87.665680
75%,1.073866e+07,1713.000000,17.000000,34.000000,56.000000,1.176369e+06,1.908781e+06,2016.000000,41.905468,-87.628056
max,1.209560e+07,2535.000000,31.000000,50.000000,77.000000,1.205119e+06,1.951535e+06,2018.000000,42.022671,-87.524529


In [17]:
sample_crime_data.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location', 'Crime'],
      dtype='object')

## Narrowed dataset to only required columns

In [18]:
final_crime_data = sample_crime_data.filter(['ID','Date','Primary Type', 'Latitude','Longitude','Year' ], axis=1)
final_crime_data.describe()


,ID,Latitude,Longitude,Year
count,5.000000e+05,495531.000000,495531.000000,500000.000000
mean,9.736921e+06,41.841988,-87.671738,2014.271508
std,1.170559e+06,0.090200,0.062937,2.329952
min,1.973600e+04,36.619446,-91.686566,2011.000000
25%,8.788834e+06,41.768131,-87.714524,2012.000000
50%,9.747599e+06,41.858568,-87.665680,2014.000000
75%,1.073866e+07,41.905468,-87.628056,2016.000000
max,1.209560e+07,42.022671,-87.524529,2018.000000


In [19]:
final_crime_data.rename(columns={'Primary Type': 'Crime'}, inplace=True)
final_crime_data

,ID,Date,Crime,Latitude,Longitude,Year
522244,10298173,10/22/2012 12:01:00 AM,DECEPTIVE PRACTICE,41.905908,-87.674277,2012
6784828,9383887,11/08/2013 10:00:00 AM,BURGLARY,41.753585,-87.563194,2013
613539,10456276,03/17/2016 08:00:00 PM,CRIMINAL DAMAGE,41.939216,-87.746878,2016
1319402,11506866,11/12/2018 08:00:00 AM,THEFT,41.754034,-87.587036,2018
6545627,8997819,11/22/2012 12:01:00 AM,OTHER OFFENSE,41.966637,-87.707576,2012
...,...,...,...,...,...,...
913411,10939452,05/09/2017 10:30:00 AM,OTHER OFFENSE,41.707711,-87.620133,2017
6968680,9697716,07/11/2014 02:00:00 PM,BURGLARY,41.764210,-87.649242,2014
7005165,9756854,08/28/2014 10:30:00 PM,ASSAULT,41.791061,-87.623196,2014
941311,10981804,06/14/2017 03:30:00 PM,THEFT,41.917025,-87.687735,2017


In [24]:
final_crime_data = final_crime_data.sample(n = 250000) 

In [25]:
# Create the connection using the imported create_engine function and 
# then invoking the connect method on it.
engine = create_engine('sqlite:///CrimeChicago.db', echo=True)
sqlite_connection = engine.connect()


2020-07-15 02:20:30,992 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 02:20:30,993 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 02:20:30,995 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 02:20:30,996 INFO sqlalchemy.engine.base.Engine ()


In [26]:
sqlite_table = "criminals_mappings00"
final_crime_data.to_sql(sqlite_table, sqlite_connection, if_exists='fail')
sqlite_connection.close()

2020-07-15 02:20:31,990 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("criminals_mappings00")
2020-07-15 02:20:31,991 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 02:20:31,992 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("criminals_mappings00")
2020-07-15 02:20:31,994 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 02:20:31,997 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE criminals_mappings00 (
	"index" BIGINT, 
	"ID" BIGINT, 
	"Date" TEXT, 
	"Crime" TEXT, 
	"Latitude" FLOAT, 
	"Longitude" FLOAT, 
	"Year" BIGINT
)


2020-07-15 02:20:31,998 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 02:20:32,141 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-15 02:20:32,142 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_criminals_mappings00_index ON criminals_mappings00 ("index")
2020-07-15 02:20:32,143 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 02:20:32,233 INFO sqlalchemy.engine.base.Engine COMMIT
2020-07-15 02:20:32,383 INFO sqlalchemy.engine.base.

### Confirm data has been added by querying the customer_location table

In [27]:
pd.read_sql_query('select * from criminals_mappings00', con=engine).head()

2020-07-15 02:20:40,088 INFO sqlalchemy.engine.base.Engine select * from criminals_mappings00
2020-07-15 02:20:40,089 INFO sqlalchemy.engine.base.Engine ()


,index,ID,Date,Crime,Latitude,Longitude,Year
0,6160381,8385872,11/30/2011 11:00:00 PM,THEFT,41.800581,-87.655036,2011
1,6722963,9283200,08/26/2013 09:30:00 PM,THEFT,41.966735,-87.695308,2013
2,497224,10255249,09/29/2015 11:15:00 AM,MOTOR VEHICLE THEFT,41.763887,-87.566476,2015
3,6021141,8168610,07/18/2011 02:50:00 PM,ASSAULT,41.865626,-87.709304,2011
4,6288127,8590583,04/30/2012 11:00:00 AM,CRIMINAL DAMAGE,41.926877,-87.735419,2012
